## Discrete Event Simulation Example - 3

## Bank  with 5 tellers  and 1 queue 


### Scenario:

  * The bank has 5 tellers and 1 queue.
  * The branch opens at 9 am and closes at 5 pm. but stays open until all customers at bank at 5pm have been served.
  * Assume that customers arrive in accordance with poisson process at rate 1 / 1.2 per minute (i.e. IID exponential interarrival times with mean  1.2 minutes)
  * service time of the tellers are IID. exponential random variables with mean 4 minutes.
  * Customers are served in a FIFO manner.
  

In [44]:
import random
import simpy

RANDOM_SEED = 42
INTERVAL_SERVICE = 4  # serve customers roughly every 4 minutes
INTERVAL_CUST_ARRIVAL = 1.2  # Generate new customers roughly every 1.2 minutes
NUM_TELLERS = 5  # number of tellers
OPEN_TIME_SPAN = 8 * 60 # total opening time in minutes

class SimStats:
    def __init__(self):
        self.total_num_served = 0
        self.W_total_sys_wait_time = 0
        self.D_total_delay_in_Q = 0
        self.Q_num_cust_in_Que = 0
        self.L_num_cust_in_Sys = 0
        self.ratio_cust_delay = 0
        self.finish_time = 0

def CustomerGenerator(env, interval, tellers, stats):
    """This generates customers randomly"""
    while env.now < OPEN_TIME_SPAN:
        cust_arrival = random.expovariate(1/interval)
        yield env.timeout(cust_arrival)       
        c = Customer(env, tellers, stats)
        env.process(c)


def Customer(env, tellers, stats):
    """Customer arrives, is served and leaves."""
    arrive = env.now
    #print('%7.4f : tellers.count %7.4f :'%(env.now, tellers.count))
    with tellers.request() as req:
        yield req 
        wait = env.now - arrive    
        stats.D_total_delay_in_Q += wait
        service = random.expovariate(1/INTERVAL_SERVICE)
        stats.total_num_served += 1
        #print('%7.4f : Finished service' % (env.now))
        yield env.timeout(service)
        stats.W_total_sys_wait_time += wait + service
      
        
def BankControl(env, tellers, finish_event, stats):
    while True:
        time_span = 2
        yield env.timeout(time_span)
        stats.Q_num_cust_in_Que += time_span * len(tellers.queue)
        stats.L_num_cust_in_Sys += time_span * (len(tellers.queue) + tellers.count)
        if (env.now >= OPEN_TIME_SPAN and tellers.count == 0 and len(tellers.queue)  == 0):
            print('%7.4f: now Finished simulation' % (env.now))
            stats.finish_time = env.now 
            yield finish_event.succeed()            
        else: 
            if (env.now >= OPEN_TIME_SPAN and (tellers.count > 0 or len(tellers.queue)  > 0)):
                for i in range(tellers.count):
                    env.step()
                stats.finish_time = env.now 
                yield finish_event.succeed()
            
    


In [45]:
# helper classes
# implement a logger
import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logging.getLogger().setLevel('INFO')

# Setup and start the simulation
logging.info('Bank Example')
random.seed(RANDOM_SEED)


logging.info("{0:16s} {1:14s} {2:14s} {3:16s} {4:16s} {5:16s} {6:16s}".format("\
    Replication", "Num Served", "Finish Time", "Avg Delay que(D)", "Avg Que Length(Q)", "Avg wait in Sys(W)", "Avg Cust in Sys(L)"))
for i in range(10):
    env = simpy.Environment()
    # Start processes and run
    simStats = SimStats()
    Tellers = simpy.Resource(env, capacity=4)
    Finish_event = env.event()
    env.process(CustomerGenerator(env, INTERVAL_CUST_ARRIVAL, Tellers, simStats))
    env.process(BankControl(env, Tellers, Finish_event, simStats))
    env.run(until = Finish_event)

    AVG_DELAY_IN_Q = simStats.D_total_delay_in_Q / simStats.total_num_served
    AVG_W_TIME_SYS = simStats.W_total_sys_wait_time / simStats.total_num_served
    AVG_Q_LENGTH = simStats.Q_num_cust_in_Que /  simStats.finish_time    
    AVG_L_NUM_SYS = simStats.L_num_cust_in_Sys / simStats.finish_time
    logging.info ("{0:9d} {1:14.2f} {2:14.2f} {3:17.2f} {4:16.2f} {5:17.2f} {6:16.2f}".format(i, simStats.total_num_served, \
        simStats.finish_time / 60, AVG_DELAY_IN_Q, AVG_Q_LENGTH, AVG_W_TIME_SYS, AVG_L_NUM_SYS))
    #SimState.reset_state()
    

INFO:Bank Example
INFO:    Replication  Num Served     Finish Time    Avg Delay que(D) Avg Que Length(Q) Avg wait in Sys(W) Avg Cust in Sys(L)
INFO:        0         454.00           8.01             16.07            15.27             20.09            19.18
INFO:        1         461.00           8.01              7.89             7.56             11.63            11.25
INFO:        2         460.00           8.01             10.83            11.18             14.51            14.94
INFO:        3         435.00           8.01             13.01            13.79             16.87            17.57
INFO:        4         489.00           8.00              6.44             6.72             10.06            10.46
INFO:        5         466.00           8.00              8.50             8.78             12.34            12.70
INFO:        6         437.00           8.02             27.18            27.87             31.03            31.75
INFO:        7         466.00           8.00        